# Create a Phovea Import Configuration

*Author: Alexander Lex; alex@sci.utah.edu *

This script creates an index.json file containing configurations to load/parse all the CSV files stored in the path specified below.


In [1]:
import pandas as pd
import numpy as np
import json
import math
from os import listdir
from os.path import isfile, join

## Importing files as DFs

In [2]:
path = "data/"
# Set the delimiter to what's used in the file
delimiter = "\t"

files = [f for f in listdir(path) if (isfile(join(path, f)) and ".csv" in f)]
print(files)


['number_one_artists.csv']


In [3]:
file_map = {}
for file in files:
    file_map[file] = pd.read_csv(join(path, file), delimiter=delimiter)
    print(file)
    #print(file_map[file].columns.values)
    print(file_map[file].dtypes)

number_one_artists.csv
ID                                     object
artist                                 object
origin                                 object
career status                          object
gender                                 object
first album (year)                      int64
studio albums (count)                   int64
year of birth                         float64
age (years)                           float64
age at first album (years)            float64
start of career (year)                  int64
end of career (year)                  float64
in business (years)                   float64
in business at first album (years)      int64
dtype: object


## Manual settings

There are some things that the script can't guess: 

In [4]:
# Can't distinguish some ID columns from ints
idtypes = {"ID", "STATENUM", "KindredID", "RelativeID", "LabID", "MaID", "PaID", "OMEDID", "ArchivePersonID"}
# If a column has more than this many labels, we consider it text, not a category
categorical_label_threshold = 10
# If you want to specify a range manually
man_range = {}
man_range["FirstBMI"] = [15,45]
man_range["MaxBMI"] = [15,45]

## Writing Configuration

In [5]:
def createColumnConfiguration(df):
    columns = []
    for column_name in df.columns.values[1:]:
        #print(column_name)
        column = df[column_name]
        col_desc = {}
        columns.append(col_desc)
        col_desc["name"] = column_name
        value = {}
        col_desc["value"] = value
        # IDTypes have to be listed manually
        if(column_name in idtypes):
            value["type"] = "idtype";
        # Because of missing values, pandas treats all numericals as floats
        elif(column.dtype == "float64" or column.dtype == "int64"):
            col_sum = column.sum()
            # Check whether it's actually an integer
            if(math.isclose(round(col_sum), col_sum, rel_tol=0.0001)):            
                value["type"] = "int"
            else:
                value["type"] = "real"
            range = []
            # Check for manually defined ranges
            if(column_name in man_range):
                range = man_range[column_name]
            else:
                #column.min()
                # range stats at 0 goes to max
                range.append(0)
                range.append(float(column.max()))
            value["range"] = range
        # If more than threshold unique values, we treat it as string
        elif((len(column.unique()) > categorical_label_threshold)):
            value["type"] = "string"
        else:
            value["type"] = "categorical"
            categories = []
            value["categories"] = categories;
            for category in column.unique():
                isNaN = False;
                try:
                    isNaN = np.isnan(category)
                    #print("Nan here", isNan, category)
                except TypeError:
                    pass
                
                if(isNaN):
                    continue
                
                category_desc = {}
                
                if(type(category) == bool):
                    if(category):
                        category_desc["name"] = "TRUE"
                    else:
                        category_desc["name"] = "FALSE"
                else:
                    category_desc["name"] = category
                category_desc["color"] = "red"
                categories.append(category_desc)
    return columns;    
        
    #print(json.dumps(columns, separators=(',', ':')))
        
        

In [6]:
import_template = """
{"name": "",
  "type": "table",
  "id": "",
  "separator": ",",
  "quotechar": "",
  "description": "autogenerated",
  "creator": "autogenerated",
  "path": "",
  "idcolumn": 0,
  "idtype": "",
  "size": [],
  "columns": []
}"""


In [7]:
def createJsonForFile(filename):
    df = file_map[filename]
    columns = createColumnConfiguration(df)
    json_config = json.loads(import_template)
    json_config["separator"] = delimiter
    json_config["quotechar"] = '"'
    json_config["columns"] = columns
    name = filename.split('.')[0]
    json_config["id"] = name
    json_config["path"] = filename
    json_config["name"] = name
    json_config["idtype"] = df.columns.values[0]
    size = [df.shape[0],df.shape[1]-1]
    json_config["size"] = size
    return json_config

In [8]:
configs = []
for k, v in file_map.items():
    json_config = createJsonForFile(k);
    # print(json_config)
    configs.append(json_config)
    

json_dump = json.dumps(configs, separators=(',', ':'), sort_keys=True, indent=2)
f = open(path+"index.json", 'w')
f.write(json_dump)
print(json_dump)



[
  {
    "columns":[
      {
        "name":"artist",
        "value":{
          "type":"string"
        }
      },
      {
        "name":"origin",
        "value":{
          "categories":[
            {
              "color":"red",
              "name":"Barbados"
            },
            {
              "color":"red",
              "name":"US"
            },
            {
              "color":"red",
              "name":"UK"
            },
            {
              "color":"red",
              "name":"Ireland"
            },
            {
              "color":"red",
              "name":"Sweden"
            }
          ],
          "type":"categorical"
        }
      },
      {
        "name":"career status",
        "value":{
          "categories":[
            {
              "color":"red",
              "name":"active"
            },
            {
              "color":"red",
              "name":"inactive"
            }
          ],
          "type":"categorical"
     